# AstroMineiros — Protótipo de ML para Detecção de Exoplanetas

Este notebook é um **protótipo reprodutível** que:
- tenta baixar light curves (TESS/Kepler) usando `lightkurve` (caso haja internet no ambiente onde rodar),
- faz pré-processamento básico (remoção de NaNs, flatten/detrend),
- gera *global view* e *local view* (binned folded arrays),
- treina um pequeno modelo CNN 1D sobre dados sintéticos caso os downloads falhem,
- salva o modelo treinado.

**Como usar**
1. Rode as células na ordem.
2. Se você estiver em um ambiente com internet (Colab, sua máquina), `lightkurve` poderá baixar dados reais.
3. Caso contrário, o notebook gerará dados sintéticos para demonstração.

> Requisitos (recomendo criar um virtualenv e instalar):
```
pip install lightkurve astropy numpy pandas scipy tensorflow scikit-learn matplotlib
```


In [ ]:
# Se estiver em um ambiente limpo, remova o comentário e instale dependências:
# !pip install lightkurve astropy numpy pandas scipy tensorflow scikit-learn matplotlib

# Importações principais
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path


In [ ]:
# Tentativa de usar lightkurve. Se não estiver disponível ou sem internet, usaremos dados sintéticos.
try:
    import lightkurve as lk
    HAS_LIGHTKURVE = True
    print('lightkurve disponível — tentaremos baixar LC reais quando possível.')
except Exception as e:
    HAS_LIGHTKURVE = False
    print('lightkurve não disponível ou falhou. Usaremos dados sintéticos. Erro:', e)

In [ ]:
from scipy.signal import savgol_filter
from astropy.timeseries import BoxLeastSquares

def detrend_time_flux(time, flux, window=401):
    # Simple Savitzky-Golay detrend as fallback (expects roughly evenly spaced time)
    # If flux is pandas Series / numpy array
    mask = ~np.isnan(flux)
    if mask.sum() < 10:
        return time, flux
    f = flux.copy()
    try:
        trend = savgol_filter(f[mask], 101, 3)  # window, polyorder
        f[mask] = f[mask] / trend
    except Exception:
        f = f / np.nanmedian(f)
    return time, f

def fold_and_bin(time, flux, period, t0, nbins=400, window=0.1):
    # fold time around period, then bin into nbins across [-0.5,0.5] phase
    phase = ((time - t0 + 0.5*period) % period) / period - 0.5
    # select window around transit for local view
    idx = np.where(np.abs(phase) < 0.5)[0]
    ph = phase[idx]
    fl = flux[idx]
    bins = np.linspace(-0.5, 0.5, nbins+1)
    inds = np.digitize(ph, bins) - 1
    binned = np.array([np.nanmedian(fl[inds==i]) if np.any(inds==i) else np.nan for i in range(nbins)])
    # simple interpolation for NaNs
    nans = np.isnan(binned)
    if nans.any():
        not_nans = ~nans
        binned[nans] = np.interp(np.flatnonzero(nans), np.flatnonzero(not_nans), binned[not_nans])
    return binned


In [ ]:
def load_example_lightcurves(max_targets=5):
    lcs = []
    if HAS_LIGHTKURVE:
        # exemplo de TICs públicos; substitua por IDs de interesse ou use search
        example_tics = [271645300, 25155310, 38846515, 25155312, 123456789]
        for tic in example_tics[:max_targets]:
            try:
                lcf = lk.search_lightcurvefile(f"TIC {tic}", mission='TESS').download()
                lc = lcf.PDCSAP_FLUX.remove_nans()
                time = lc.time.value
                flux = lc.flux.value / np.nanmedian(lc.flux.value)
                lcs.append({'id':f'TIC{tic}','time':time,'flux':flux})
                print('baixado:', tic)
            except Exception as e:
                print('falha ao baixar', tic, e)
    # Se não houver reais (ou falhar), gerar sintéticos
    if len(lcs) == 0:
        print('Gerando dados sintéticos de exemplo...')
        rng = np.random.RandomState(42)
        for i in range(max_targets):
            time = np.linspace(0,27.4,3000)  # ~TESS sector length sample
            flux = 1.0 + 0.0005*rng.randn(len(time))
            # inject synthetic transit
            period = rng.uniform(1.5,12.0)
            t0 = rng.uniform(0,period)
            depth = rng.uniform(0.001,0.02)
            dur = rng.uniform(0.05,0.2)  # in phase units fraction of period
            phase = ((time - t0 + 0.5*period) % period) / period - 0.5
            in_transit = np.abs(phase) < (dur/2)
            flux[in_transit] -= depth
            lcs.append({'id':f'SYN{i}','time':time,'flux':flux,'period':period,'t0':t0,'depth':depth})
        print('Criados', len(lcs), 'light curves sintéticas.')
    return lcs

lcs = load_example_lightcurves(max_targets=5)


In [ ]:
processed = []
for lc in lcs:
    time = np.array(lc['time'])
    flux = np.array(lc['flux'])
    time, flux = detrend_time_flux(time, flux)
    # Run a quick BLS to get candidate period (only if enough points)
    try:
        bls = BoxLeastSquares(time, flux)
        periods = np.linspace(0.5, 20, 2000)
        res = bls.power(periods, 0.1)
        best = np.argmax(res.power)
        period = res.period[best]
        t0 = res.transit_time[best]
    except Exception:
        period = lc.get('period', 5.0)
        t0 = lc.get('t0', 0.1)
    global_view = fold_and_bin(time, flux, period, t0, nbins=400)
    local_view = fold_and_bin(time, flux, period, t0, nbins=200)
    processed.append({'id':lc['id'],'period':period,'t0':t0,'global':global_view,'local':local_view})
print('Processados', len(processed), 'curvas.')

In [ ]:
# Plot a global + local for the first LC
import matplotlib.pyplot as plt
item = processed[0]
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(np.linspace(-0.5,0.5,400), item['global'])
plt.title(f"Global view — {item['id']} (P={item['period']:.3f} d)")
plt.xlabel('Phase')
plt.subplot(1,2,2)
plt.plot(np.linspace(-0.5,0.5,200), item['local'])
plt.title('Local view (zoom)')
plt.xlabel('Phase')
plt.tight_layout()
plt.show()

In [ ]:
# Build X arrays (stack global+local) and synthetic labels
X = []
y = []
for i,p in enumerate(processed):
    g = p['global']
    l = p['local']
    arr = np.concatenate([g, l])  # simple concat; shape (600,)
    X.append(arr)
    # If original had depth field (synthetic), label as planet (1), else random
    if p['id'].startswith('SYN'):
        y.append(1)  # planet
    else:
        # for real LCs we don't have labels here — mark as unknown (0)
        y.append(0)
X = np.array(X)
y = np.array(y)
print('X shape', X.shape, 'y shape', y.shape)

In [ ]:
# Small TF model (1D conv) - works with small sample for demo purposes
import tensorflow as tf
from tensorflow.keras import layers, models

X_in = X[..., np.newaxis]  # add channel
inp = layers.Input(shape=(X_in.shape[1],1))
x = layers.Conv1D(16,7,activation='relu',padding='same')(inp)
x = layers.MaxPooling1D(4)(x)
x = layers.Conv1D(32,5,activation='relu',padding='same')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(32,activation='relu')(x)
out = layers.Dense(2,activation='softmax')(x)

model = models.Model(inp,out)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

# For demo, convert labels to two classes (planet vs non-planet)
y_bin = (y>0).astype(int)
if len(X_in) >= 2:
    model.fit(X_in, y_bin, epochs=12, batch_size=2, verbose=2)
else:
    print('Poucos exemplos — pulando treinamento real; mostrando previsão com pesos aleatórios.')
    preds = model.predict(X_in)
    print('Preds:', preds)

In [ ]:
# Salvar modelo para uso posterior
outdir = Path('models')
outdir.mkdir(exist_ok=True)
model_path = outdir / 'astro_mineiros_demo_model'
try:
    model.save(model_path)
    print('Modelo salvo em', model_path)
except Exception as e:
    print('Falha ao salvar modelo:', e)

## Próximos passos sugeridos

- Substituir/rodar as células de ingestão por queries específicas do **NASA Exoplanet Archive** ou MAST para baixar KOIs/TESS candidates reais.
- Expandir dataset e rotulagem usando tabelas oficiais (Confirmed / Candidate / False Positive).
- Implementar pipeline de treino/validação robusta (split por estrela, k-fold por campanha).
- Construir API (FastAPI) e front-end (React) conforme o esqueleto proposto na documentação do projeto.

Se quiser, eu posso:
- Gerar o **esqueleto do repositório** (FastAPI + React + Docker) ao lado deste notebook, pronto para deploy,
- Ou customizar este notebook para usar somente TESS/Kepler com exemplos de queries TAP.

Diga qual você prefere que eu gere a seguir: o esqueleto do repositório completo (API + frontend + Docker) **ou** eu customizo este notebook para baixar apenas TESS (e incluir processamento de centroid shifts, vetores de co-trending, etc.).